In [2]:
from google.colab import files
uploaded = files.upload()


Saving dataset.csv to dataset.csv


In [7]:
import pandas as pd
df = pd.read_csv('dataset.csv')


In [9]:


# Required Libraries
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
output_notebook()

# ----------------------
# Project Explanation Section
# ----------------------
print("""
🔍 Project Explanation

📌 Demand Function:
We define demand using a weighted sum of key influencing factors:
    Demand = α × (Occupancy / Capacity)
           + β × QueueLength
           - γ × Traffic
           + δ × IsSpecialDay
           + ε × VehicleTypeWeight

Where vehicle weights are: car=1.0, bike=0.5, truck=1.5.
This function captures how occupancy, congestion, and event conditions influence parking demand.

📌 Assumptions:
- Traffic is mapped numerically (low=1, medium=5, high=10).
- Special days increase parking demand.
- Different vehicles show different levels of price sensitivity.
- Price should remain bounded (0.5×base to 2×base) and change smoothly over time.

📌 Pricing Behavior:
- As demand increases (e.g., more vehicles, queues, special events), the price increases.
- If competitor lots are cheaper and this lot is full → our price is reduced to reroute demand.
- If competitor lots are expensive → we slightly increase price (while staying attractive).

📈 Visualization:
We use Bokeh to plot real-time pricing and demand trends:
- Price ($) vs Time
- Normalized Demand vs Time
These plots justify the pricing logic and show how external factors impact pricing dynamically.
""")

# ----------------------
# Load Dataset
# ----------------------
df = pd.read_csv('dataset.csv')

# Preprocessing
traffic_map = {'low': 1, 'medium': 5, 'high': 10}
df['Traffic'] = df['TrafficConditionNearby'].map(traffic_map)
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True, errors='coerce')
df.dropna(subset=['Timestamp'], inplace=True)
df.sort_values(by='Timestamp', inplace=True)

# Filter for one parking lot (for simplicity)
lot = df[df['SystemCodeNumber'] == df['SystemCodeNumber'].iloc[0]].reset_index(drop=True)

# ----------------------
# Utility Functions
# ----------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# ----------------------
# Model 1: Baseline Pricing
# ----------------------
def baseline_price(prev_price, occupancy, capacity, alpha=1.0):
    utilization = occupancy / (capacity + 1e-6)
    new_price = prev_price + alpha * utilization
    return max(0.5 * 10, min(2 * 10, new_price))

# ----------------------
# Model 2: Demand-Based Pricing
# ----------------------
def compute_demand(occupancy, capacity, queue_length, traffic, special_day, vehicle_type, weights):
    vehicle_weight = {"car": 1.0, "bike": 0.5, "truck": 1.5}.get(vehicle_type, 1.0)
    occ_ratio = occupancy / (capacity + 1e-6)
    return (weights['alpha'] * occ_ratio +
            weights['beta'] * queue_length -
            weights['gamma'] * traffic +
            weights['delta'] * special_day +
            weights['epsilon'] * vehicle_weight)

def demand_price(base_price, demand, demand_min, demand_max, lambd=0.5):
    norm_demand = (demand - demand_min) / (demand_max - demand_min + 1e-6)
    new_price = base_price * (1 + lambd * norm_demand)
    return max(0.5 * base_price, min(2 * base_price, new_price))

# ----------------------
# Model 3: Competitive Pricing
# ----------------------
def competitive_adjustment(price, own_lot_full, nearby_prices):
    if own_lot_full and any(p < price for p in nearby_prices):
        return price * 0.9
    elif all(p > price for p in nearby_prices):
        return price * 1.1
    return price

# ----------------------
# Apply Model 2: Demand-Based Pricing
# ----------------------
weights = {'alpha': 0.6, 'beta': 0.2, 'gamma': 0.3, 'delta': 0.5, 'epsilon': 0.4}
demand_vals = []
price_vals = []
prev_price = 10

for i, row in lot.iterrows():
    demand = compute_demand(row['Occupancy'], row['Capacity'], row['QueueLength'],
                             row['Traffic'], row['IsSpecialDay'], row['VehicleType'], weights)
    demand_vals.append(demand)
    price = demand_price(10, demand, min(demand_vals), max(demand_vals))
    price_vals.append(price)

lot['Demand'] = demand_vals
lot['Price'] = price_vals

# ----------------------
# Bokeh Visualization
# ----------------------
p = figure(x_axis_type='datetime', title="Real-Time Pricing for Parking Lot", width=800, height=300)
p.line(lot['Timestamp'], lot['Price'], legend_label="Price ($)", line_width=2, color="navy")
p.line(lot['Timestamp'], lot['Demand'], legend_label="Demand", line_width=2, color="green")
p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price / Demand"

show(column(p))



🔍 Project Explanation

📌 Demand Function:
We define demand using a weighted sum of key influencing factors:
    Demand = α × (Occupancy / Capacity)
           + β × QueueLength
           - γ × Traffic
           + δ × IsSpecialDay
           + ε × VehicleTypeWeight

Where vehicle weights are: car=1.0, bike=0.5, truck=1.5.
This function captures how occupancy, congestion, and event conditions influence parking demand.

📌 Assumptions:
- Traffic is mapped numerically (low=1, medium=5, high=10).
- Special days increase parking demand.
- Different vehicles show different levels of price sensitivity.
- Price should remain bounded (0.5×base to 2×base) and change smoothly over time.

📌 Pricing Behavior:
- As demand increases (e.g., more vehicles, queues, special events), the price increases.
- If competitor lots are cheaper and this lot is full → our price is reduced to reroute demand.
- If competitor lots are expensive → we slightly increase price (while staying attractive).

📈 Visualizatio